In [ ]:
%matplotlib inline
import flopy
import matplotlib.pyplot as plt
import matplotlib.tri as tri
import numpy as np
import pandas as pd
import pathlib as pl
import pickle
import xarray as xa

In [ ]:
sample_frequency = "monthly"  # monthly or annual
useET = True

In [ ]:
in2ft = 1.0 / 12.0

## Load the appropriate temporal data

In [ ]:
idx_end_calibration = 0
if sample_frequency == "monthly":
    idx_end_period2 = 120
    idx_end_period3 = 240
elif sample_frequency == "annual":
    idx_end_period2 = 10
    idx_end_period3 = 20
else:
    raise ValueError(f"invalid sample_frequency: '{sample_frequency}'")

In [ ]:
path = pl.Path(f"../data/temporal_data_{sample_frequency}.parquet")
temporal_df = pd.read_parquet(path)

In [ ]:
temporal_df

## Define the stress period data

In [ ]:
start_date = pd.to_datetime("1962-01-01 00:00:00")
start_date_time = str(start_date).replace(" ", "T")

end_calibration = temporal_df.index[idx_end_calibration]
end_period_two = temporal_df.index[idx_end_period2]
end_period_three = temporal_df.index[idx_end_period2]

end_periods = [end_calibration, end_period_two, end_period_three]
end_periods

In [ ]:
totim_end = [float((end_calibration - start_date).days)]
totim_end += [float((end_period_two - start_date).days)]
totim_end += [float((end_period_three - start_date).days)]
totim_end

In [ ]:
perlen = temporal_df["perlen"].values
nstp = [1 for idx in range(len(perlen))]
tsmult = [1.0 for idx in range(len(perlen))]

nper = len(perlen)
tdis_ds = [(p, n, t) for p, n, t in zip(perlen, nstp, tsmult)]

## Spatial data for the model

In [ ]:
nc_path = pl.Path("../data/synthetic_valley_truth.nc")
nc_ds = xa.open_dataset(nc_path)
lake_location = nc_ds["lake_location"].to_numpy()

## Create the model

In [ ]:
name = "sv"
ws = pl.Path(f"../../models/synthetic-valley-base-{sample_frequency}")

## Define discretization

In [ ]:
top = nc_ds["top_layer1"].values + 2
top[nc_ds["lake_location"].values == 1] = 20

In [ ]:
nlay = 5
nrow, ncol = top.shape
nlay, nrow, ncol

In [ ]:
shape3d = (nlay, nrow, ncol)
shape2d = (nrow, ncol)

In [ ]:
idomain = [1] * nlay

In [ ]:
delr = delc = 500.0

In [ ]:
botm = [
    nc_ds["bottom_layer1"].values,
    nc_ds["bottom_layer2"].values,
    nc_ds["bottom_layer3"].values,
    nc_ds["bottom_layer4"].values,
    nc_ds["bottom_layer5"].values,
]

In [ ]:
elevations = np.array([top] + botm)
thickness = np.array([elevations[k] - elevations[k + 1] for k in range(nlay)])

In [ ]:
lake_location = nc_ds["lake_location"].values

## Define starting heads

In [ ]:
strt = [
    nc_ds["head_layer1"].values,
    nc_ds["head_layer2"].values,
    nc_ds["head_layer3"].values,
    nc_ds["head_layer4"].values,
    nc_ds["head_layer5"].values,
]

## Define aquifer properties 

In [ ]:
kvkh_ratio = 0.4

In [ ]:
lake_location = nc_ds["lake_location"].to_numpy()

In [ ]:
clay_location = nc_ds["clay_location"].to_numpy()

In [ ]:
kh3 = nc_ds["clay_kv"].to_numpy()
kh3[clay_location == 0] = nc_ds["hyat"].values[clay_location == 0]

In [ ]:
kv3 = nc_ds["clay_kv"].to_numpy()
kv3[clay_location == 0] = nc_ds["hyat"].values[clay_location == 0] * kvkh_ratio

In [ ]:
k11 = np.array([nc_ds["hyat"].to_numpy() for klay in range(nlay)])
k33 = np.array([nc_ds["hyat"].to_numpy() * kvkh_ratio for k in range(nlay)])

In [ ]:
plt.imshow(kv3)

In [ ]:
transmissitivty = k11 * thickness

In [ ]:
k11[2, :, :] = kh3
k33[2, :, :] = kv3

In [ ]:
k33 /= k11
kv3 = k33[2, :, :]
kv3[clay_location == 0] = 1.0
kv3[clay_location == 1] = kvkh_ratio

In [ ]:
plt.imshow(k33[2, :, :])

In [ ]:
k1 = k11[0, :, :]
k1[lake_location == 1] = 2000000.0
k11[0, :, :] = k1

In [ ]:
k1 = k33[0, :, :]
k1[lake_location == 1] = 2000000.0
k33[0, :, :] = k1

In [ ]:
ss = [5e-5, 5e-5, 5e-3, 5e-5, 5e-5]
v = np.full(shape2d, 5e-5, dtype=float)
v[clay_location == 1] = 5e-3
ss[2] = v


sy = [0.25, 0.25, 0.025, 0.25, 0.25]
v = np.full(shape2d, 0.25, dtype=float)
v[lake_location == 1] = 1.0
sy[0] = v

v = np.full(shape2d, 0.25, dtype=float)
v[clay_location == 1] = 0.45
sy[2] = v

## Define the recharge

In [ ]:
recharge_spd = {}
for n in range(nper):
    row = temporal_df.iloc[n]
    spd = []
    for i in range(nrow):
        for j in range(ncol):
            if lake_location[i, j] == 1:
                if useET:
                    rch_ts = "PRCP (Inches)"
                    boundname = "lake"
                else:
                    rch_ts = "netrch lake (inches)"
                    boundname = "lake"
            else:
                rch_ts = "netrch land (inches)"
                boundname = "land"
            spd.append(((0, i, j), float(row[rch_ts]) * in2ft, boundname))
    recharge_spd[n] = spd

## Define the evt

In [ ]:
if useET:
    nlake_cells = 0
    lake_evt_spd = {}
    evt_ts = "lake et (inches)"
    for n in range(nper):
        row = temporal_df.iloc[n]
        spd = []
        for i in range(nrow):
            for j in range(ncol):
                if lake_location[i, j] == 1:
                    spd.append(((0, i, j), -4.0, float(row[evt_ts]) * in2ft, 2.0))
        lake_evt_spd[n] = spd
    nlake_cells = len(spd)

## Define the river

In [ ]:
nriv = 18
stage = np.linspace(1.75, 0.05, nriv)
stage

In [ ]:
riv_bndname = ["RIV" for i in range(nriv)]

In [ ]:
river_spd = [
    ((0, i + 22, 8), float(stage[i]), 1e5, float(stage[i]) - 1.0, riv_bndname[i])
    for i in range(nriv)
]
river_spd

## Define the spring

In [ ]:
spring_cellids = [(3, 22, 7), (4, 22, 7)]
spring_elev = river_spd[0][3]
spring_spd = []
radius = 25.0
for cellid in spring_cellids:
    b = thickness[cellid]
    cond = 1000.0 * k11[cellid] * float(np.pi * radius**2) / (0.5 * delr)
    spring_spd.append((cellid, float(spring_elev), cond, "leake_spring"))
spring_spd = {0: spring_spd}

## Define the coastal GHBs

In [ ]:
skip_cellids = [(0, 39, 8)]
sw_head = river_spd[-1][1]
rho = 1025.0
rhof = 1000.0
rho_rhof = rho / rhof
rhod_rhof = (rho - rhof) / rhof
ghb_spd = []
for k in range(nlay):
    for j in range(ncol):
        cellid = (k, nrow - 1, j)
        cellid1 = (k + 1, nrow - 1, j)
        if cellid in skip_cellids:
            continue
        z0 = min(elevations[cellid], sw_head)
        z1 = min(elevations[cellid1], sw_head)
        z = 0.5 * (z0 + z1)
        b = z0 - z1
        cond = k11[cellid] * b * delc / (0.5 * delr)
        hf = sw_head * rho_rhof - rhod_rhof * z
        ghb_spd.append((cellid, float(hf), float(cond)))
ghb_spd

## Define the wells

In [ ]:
def get_well_data(i, j, tag, k0=3, k1=4):
    T = 0.0
    T_k = []
    for k in range(k0, k1 + 1):
        value = float(transmissitivty[k, i, j])
        T += value
        T_k.append(value)
    well_data = []
    fractions = []
    idx = 0
    for k in range(k0, k1 + 1):
        well_data.append((k, i, j, f"{tag}_{k + 1}", tag))
        fractions.append(T_k[idx] / T)
        idx += 1
    return well_data, fractions

In [ ]:
well_spd_base = {
    1: [
        (5, 14, "reilly"),
        (32, 5, "vc"),
    ],
}

In [ ]:
well_fractions = {}
for n, values in well_spd_base.items():
    rates = []
    fractions = []
    for i, j, well_name in values:
        rate, fraction = get_well_data(i, j, well_name)
        rates += rate
        fractions += fraction
    for idx, (k, i, j, tag, well_name) in enumerate(rates):
        well_fractions[tag] = ((k, i, j), fractions[idx], well_name)

well_fractions

In [ ]:
well_spd = {}
for n in range(1, nper):
    row = temporal_df.iloc[n]
    spd = []
    for key, (cellid, fraction, well_name) in well_fractions.items():
        spd.append(((cellid), float(row[well_name]) * fraction, well_name))
    well_spd[n] = spd
well_spd[1]

## Define the well for the prediction

In [ ]:
prediction_spd = {}
for n in range(idx_end_period2 + 1, nper):
    # rate = 0.0
    # if n >= idx_end_period2 + 1:
    rate = -300000.0
    prediction_spd[n] = [((4, 34, 15), rate)]

## Define the head observations

In [ ]:
obs_rc_locs = [
    (2, 17),
    (3, 10),
    (6, 20),
    (12, 22),
    (14, 11),
    (16, 18),
    (17, 1),
    (18, 6),
    (19, 11),
    (18, 22),
    (26, 5),
    (27, 11),
    (28, 23),
    (30, 6),
    (33, 14),
    (36, 1),
    (37, 22),
]

In [ ]:
np.array(botm)[:, 3, 10]

In [ ]:
well_depth = [
    -200.74,
    -100.00,
    -100.00,
    -100.00,
    -224.62,
    -100.00,
    -100.00,
    -100.00,
    -100.00,
    -100.00,
    -100.00,
    -100.00,
    -100.00,
    -100.00,
    -233.22,
    -100.00,
    -100.00,
]

In [ ]:
wt_obs = []
aq_layer = []
aq_obs = []
for idx, (i, j) in enumerate(obs_rc_locs):
    iloc = (i, j)
    tag = "head_layer1"
    wt_obs.append(float(nc_ds[tag].values[iloc]))
    wz = well_depth[idx]
    zcell = np.array(botm)[:, i, j]
    klay = 0
    for kk in range(1, nlay):
        z0 = zcell[kk - 1]
        z1 = zcell[kk]
        if wz < z0 and wz >= z1:
            klay = kk
            break
    tag = f"head_layer{klay + 1}"
    aq_layer.append(klay)
    aq_obs.append(float(nc_ds[tag].values[iloc]))

In [ ]:
obs_path = pl.Path("../data")
combined_data = [obs_rc_locs, well_depth, aq_layer]
with open(obs_path / "obs_data.pkl", "wb") as f:
    pickle.dump(combined_data, f)

In [ ]:
print(wt_obs)

In [ ]:
print(aq_layer)

In [ ]:
print(aq_obs)

In [ ]:
cal_loc_wt = [(0, i, j) for i, j in obs_rc_locs]

In [ ]:
cal_loc_aq = [(aq_layer[idx], i, j) for idx, (i, j) in enumerate(obs_rc_locs)]

In [ ]:
obs_loc = {}
obs_loc[f"{name}.gwf.wt.csv"] = [
    (f"wt{idx + 1:02d}-i{i}-j{j}", "HEAD", (0, i, j))
    for idx, (i, j) in enumerate(obs_rc_locs)
]

In [ ]:
obs_loc[f"{name}.gwf.aq.csv"] = [
    (f"aq{idx + 1:02d}-i{i}-j{j}", "HEAD", (aq_layer[idx], i, j))
    for idx, (i, j) in enumerate(obs_rc_locs)
]

In [ ]:
obs_loc

In [ ]:
obs_loc[f"{name}.gwf.scenario.csv"] = [
    ("Reilly", "HEAD", (4, 5, 14)),
    ("VC", "HEAD", (4, 34, 15)),
    ("DW", "HEAD", (0, 22, 19)),
]

In [ ]:
obs_loc[f"{name}.lake.obs.csv"] = [
    ("Lake-stage", "HEAD", (0, 15, 8)),
]

In [ ]:
obs_loc

## Build the model

In [ ]:
sim = flopy.mf6.MFSimulation(sim_name=name, sim_ws=ws, write_headers=False)

In [ ]:
tdis = flopy.mf6.ModflowTdis(
    sim,
    start_date_time=start_date_time,
    time_units="DAYS",
    nper=nper,
    perioddata=tdis_ds,
)

In [ ]:
ims = flopy.mf6.ModflowIms(
    sim,
    print_option="summary",
    complexity="complex",
    under_relaxation=None,
    linear_acceleration="bicgstab",
    outer_maximum=5000,
    inner_maximum=100,
    outer_dvclose=1e-5,
    inner_dvclose=1e-6,
)

In [ ]:
gwf = flopy.mf6.ModflowGwf(
    sim, modelname=name, save_flows=True, newtonoptions="newton under_relaxation"
)

In [ ]:
dis = flopy.mf6.ModflowGwfdis(
    gwf,
    length_units="FEET",
    nlay=nlay,
    nrow=nrow,
    ncol=ncol,
    delr=delr,
    delc=delc,
    top=top,
    botm=botm,
    idomain=idomain,
)

In [ ]:
ic = flopy.mf6.ModflowGwfic(gwf, strt=strt)

In [ ]:
npf = flopy.mf6.ModflowGwfnpf(
    gwf, save_specific_discharge=True, k=k11, k33=k33, k33overk=True
)

In [ ]:
sto = flopy.mf6.ModflowGwfsto(
    gwf,
    sy=sy,
    ss=ss,
    steady_state={0: True},
    transient={1: True},
)

In [ ]:
rch = flopy.mf6.ModflowGwfrch(
    gwf,
    boundnames=True,
    maxbound=shape2d,
    stress_period_data=recharge_spd,
)

In [ ]:
if useET:
    lake_evt = flopy.mf6.ModflowGwfevt(
        gwf,
        maxbound=nlake_cells,
        stress_period_data=lake_evt_spd,
    )

In [ ]:
riv_obs = {f"{name}.riv.obs.csv": [("RIV-SWGW", "RIV", "RIV")]}

In [ ]:
riv = flopy.mf6.ModflowGwfriv(
    gwf,
    boundnames=True,
    print_flows=True,
    maxbound=nriv,
    stress_period_data=river_spd,
    observations=riv_obs,
    pname="riv-1",
)

In [ ]:
spring_obs = {f"{name}.spring.obs.csv": [("SPRING-FLOW", "DRN", "LEAKE_SPRING")]}

In [ ]:
drn_spring = flopy.mf6.ModflowGwfdrn(
    gwf,
    boundnames=True,
    pname="spring",
    maxbound=2,
    stress_period_data=spring_spd,
    observations=spring_obs,
)

In [ ]:
coastal_ghb = flopy.mf6.ModflowGwfghb(
    gwf,
    maxbound=len(ghb_spd),
    stress_period_data=ghb_spd,
)

In [ ]:
wel = flopy.mf6.ModflowGwfwel(
    gwf,
    boundnames=True,
    filename=f"{name}.pwell.wel",
    stress_period_data=well_spd,
    maxbound=4,
    pname="pwell",
)

In [ ]:
pred_well = flopy.mf6.ModflowGwfwel(
    gwf,
    filename=f"{name}.prediction.well",
    maxbound=1,
    stress_period_data=prediction_spd,
    pname="prediction",
)

In [ ]:
oc = flopy.mf6.ModflowGwfoc(
    gwf,
    head_filerecord=f"{name}.hds",
    budget_filerecord=f"{name}.cbc",
    budgetcsv_filerecord=f"{name}-budget.csv",
    saverecord=[("HEAD", "ALL"), ("BUDGET", "ALL")],
)

In [ ]:
gwf_obs = flopy.mf6.ModflowUtlobs(
    gwf,
    print_input=True,
    continuous=obs_loc,
    pname="gwf-obs",
    filename=f"{name}.gwf.obs",
)

In [ ]:
# gwf.set_all_data_external(external_data_folder="external")

sim.write_simulation()

In [ ]:
success = sim.run_simulation()
assert success, "Model did not terminate normally"

## Plot the results

### Model Properties

In [ ]:
with flopy.plot.styles.USGSMap():
    fig, axs = plt.subplots(1, 5, figsize=(9, 3), sharey=True)
    fig.suptitle("Hydraulic conductivity")

    for idx in range(nlay):
        ax = axs[idx]
        mm = flopy.plot.PlotMapView(model=gwf, layer=idx, ax=ax)
        mm.plot_array(k11[idx], masked_values=[2000000.0])
        ax.set_title(f"Layer {idx + 1}")

In [ ]:
with flopy.plot.styles.USGSMap():
    fig, axs = plt.subplots(1, 5, figsize=(9, 3), sharey=True)
    fig.suptitle("Bottom Elevation")

    for idx in range(nlay):
        ax = axs[idx]
        mm = flopy.plot.PlotMapView(model=gwf, layer=idx, ax=ax)
        mm.plot_array(botm[idx])
        ax.set_title(f"Layer {idx + 1}")

In [ ]:
with flopy.plot.styles.USGSMap():
    fig, axs = plt.subplots(1, 5, figsize=(9, 3), sharey=True)
    fig.suptitle("Cell thickness")
    z = gwf.modelgrid.cell_thickness

    for idx in range(nlay):
        ax = axs[idx]
        mm = flopy.plot.PlotMapView(model=gwf, layer=idx, ax=ax)
        mm.plot_array(z[idx])
        ax.set_title(f"Layer {idx + 1}")

### Simulated Heads and Drawdown

In [ ]:
levels = np.arange(2, 20.0, 2)

#### Calibration

In [ ]:
hds = gwf.output.head().get_data(totim=totim_end[0])

In [ ]:
with flopy.plot.styles.USGSMap():
    fig, axs = plt.subplots(1, 5, figsize=(9, 3), sharey=True)
    fig.suptitle("Heads - calibration")

    for idx in range(nlay):
        ax = axs[idx]
        mm = flopy.plot.PlotMapView(model=gwf, layer=idx, ax=ax)
        mm.plot_array(hds)
        cs = mm.contour_array(hds, colors="white", levels=levels)
        plt.clabel(cs, inline=True, fontsize=8)
        ax.set_title(f"Layer {idx + 1}")

In [ ]:
v = gwf.output.budget().get_data(text="riv", totim=totim_end[0])[0]["q"]
print(f"River infiltration: {np.all(v > 0)}\n{v}")

##### Calculate the residuals

In [ ]:
sim_wt = np.array([hds[idx] for idx in cal_loc_wt])

In [ ]:
resid_wt = sim_wt - np.array(wt_obs)
resid_wt

In [ ]:
sim_aq = np.array([hds[idx] for idx in cal_loc_aq])

In [ ]:
resid_aq = sim_aq - np.array(aq_obs)
resid_aq

In [ ]:
resid_gb = np.concatenate((resid_wt, resid_aq))

In [ ]:
print(
    f"Water Table Statistics\nMean Error: {resid_wt.mean()} ft.\nRMSE:       {np.sqrt((resid_wt**2).sum()) / resid_wt.shape[0]} ft."
)

In [ ]:
print(
    f"Lower Aquifer Statistics\nMean Error: {resid_aq.mean()} ft.\nRMSE:       {np.sqrt((resid_aq**2).sum()) / resid_aq.shape[0]} ft."
)

In [ ]:
print(
    f"Global Statistics\nMean Error: {resid_gb.mean()} ft.\nRMSE:       {np.sqrt((resid_gb**2).sum()) / resid_gb.shape[0]} ft."
)

##### Plot the residuals

In [ ]:
xy = [
    (float(gwf.modelgrid.xcellcenters[i, j]), float(gwf.modelgrid.ycellcenters[i, j]))
    for i, j in obs_rc_locs
]

In [ ]:
x, y = np.array(xy)[:, 0], np.array(xy)[:, 1]

In [ ]:
grid_x, grid_y = np.meshgrid(gwf.modelgrid.xycenters[0], gwf.modelgrid.xycenters[1])

In [ ]:
# Linearly interpolate the data (x, y) on a grid defined by (xi, yi).
triang = tri.Triangulation(x, y)

In [ ]:
interpolator = tri.LinearTriInterpolator(triang, resid_wt)
grid_resid_wt = interpolator(grid_x, grid_y)

In [ ]:
interpolator = tri.LinearTriInterpolator(triang, resid_aq)
grid_resid_aq = interpolator(grid_x, grid_y)

In [ ]:
resid_levels = np.arange(-2, 2.25, 0.25)

In [ ]:
with flopy.plot.styles.USGSMap():
    fig, axs = plt.subplots(1, 2, figsize=(8, 5), sharey=True)
    fig.suptitle("Residuals")

    ax = axs[0]
    ax.set_xlim(0, 12500)
    ax.set_ylim(0, 20000)
    mm = flopy.plot.PlotMapView(model=gwf, ax=ax, extent=gwf.modelgrid.extent)
    mm.plot_array(lake_location, cmap="Blues_r", masked_values=[0])
    mm.plot_grid(lw=0.5, color="0.5")
    ax.scatter(x, y, s=3, c="black")
    for i, txt in enumerate(resid_wt):
        ax.annotate(f"{txt:.2f}", (x[i], y[i]))
    cs = ax.contour(
        grid_x,
        grid_y,
        grid_resid_wt,
        levels=resid_levels,
        linewidths=0.75,
        colors="red",
    )
    plt.clabel(cs, inline=True, fontsize=8)
    ax.set_title("Water Table")

    ax = axs[1]
    ax.set_xlim(0, 12500)
    ax.set_ylim(0, 20000)
    mm = flopy.plot.PlotMapView(model=gwf, ax=ax, extent=gwf.modelgrid.extent)
    mm.plot_grid(lw=0.5, color="0.5")
    ax.scatter(x, y, s=3, c="black")
    for i, txt in enumerate(resid_aq):
        ax.annotate(f"{txt:.2f}", (x[i], y[i]), clip_on=False)
    cs = ax.contour(
        grid_x,
        grid_y,
        grid_resid_aq,
        levels=resid_levels,
        linewidths=0.75,
        colors="red",
    )
    plt.clabel(cs, inline=True, fontsize=8)
    ax.set_title("Lower Aquifer")

**NOTE:** There is spatial bias in the simulated results (*i.e.*, residuals are positive in the Northeast and negative in the Southwest).

#### Case A

In [ ]:
with flopy.plot.styles.USGSMap():
    fig, axs = plt.subplots(1, 5, figsize=(9, 3), sharey=True)
    fig.suptitle("Heads - Transient Period 1")
    hds = gwf.output.head().get_data(totim=totim_end[1])

    for idx in range(nlay):
        ax = axs[idx]
        mm = flopy.plot.PlotMapView(model=gwf, layer=idx, ax=ax)
        mm.plot_array(hds)
        cs = mm.contour_array(hds, colors="white", levels=levels)
        plt.clabel(cs, inline=True, fontsize=8)
        ax.set_title(f"Layer {idx + 1}")

In [ ]:
with flopy.plot.styles.USGSMap():
    fig, axs = plt.subplots(1, 5, figsize=(9, 3), sharey=True)
    fig.suptitle("Drawdown - Transient Period 1")
    ddn = gwf.output.head().get_data(totim=totim_end[0]) - gwf.output.head().get_data(
        totim=totim_end[1]
    )

    ddn_max = ddn[:, 16, :].max()

    for idx in range(nlay):
        ax = axs[idx]
        mm = flopy.plot.PlotMapView(model=gwf, layer=idx, ax=ax)
        mm.plot_array(ddn)
        cs = mm.contour_array(ddn, colors="white", levels=levels)
        plt.clabel(cs, inline=True, fontsize=8)
        ax.set_title(f"Layer {idx + 1}")

In [ ]:
print(f"Maximum Drawdown: {ddn_max}")

In [ ]:
v = gwf.output.budget().get_data(text="riv", totim=totim_end[1])[0]["q"]
print(f"Induced river infiltration: {np.all(v > 0)}\n{v}")

#### Extra Run

In [ ]:
with flopy.plot.styles.USGSMap():
    fig, axs = plt.subplots(1, 5, figsize=(9, 3), sharey=True)
    fig.suptitle("Heads - Transient Period 2")
    hds = gwf.output.head().get_data(totim=totim_end[2])

    for idx in range(nlay):
        ax = axs[idx]
        mm = flopy.plot.PlotMapView(model=gwf, layer=idx, ax=ax)
        mm.plot_array(hds)
        cs = mm.contour_array(hds, colors="white", levels=levels)
        plt.clabel(cs, inline=True, fontsize=8)
        ax.set_title(f"Layer {idx + 1}")

In [ ]:
with flopy.plot.styles.USGSMap():
    fig, axs = plt.subplots(1, 5, figsize=(9, 3), sharey=True)
    fig.suptitle("Drawdown - Transient Period 2")
    ddn = gwf.output.head().get_data(totim=totim_end[0]) - gwf.output.head().get_data(
        totim=totim_end[2]
    )

    ddn_max = ddn[:, 16, :].max()

    for idx in range(nlay):
        ax = axs[idx]
        mm = flopy.plot.PlotMapView(model=gwf, layer=idx, ax=ax)
        mm.plot_array(ddn)
        cs = mm.contour_array(ddn, colors="white", levels=levels)
        plt.clabel(cs, inline=True, fontsize=8)
        ax.set_title(f"Layer {idx + 1}")

In [ ]:
print(f"Maximum Drawdown: {ddn_max}")

In [ ]:
v = gwf.output.budget().get_data(text="riv", totim=totim_end[2])[0]["q"]
print(f"Induced river infiltration: {np.all(v > 0)}\n{v}")

### Spring flow results

In [ ]:
df = gwf.spring.output.obs().get_dataframe(start_datetime=start_date)
df["SPRING-FLOW"] /= -86400

Q0 = df["SPRING-FLOW"].iloc[0]
df["PCT_DIFF"] = -100.0 * (df["SPRING-FLOW"] - Q0) / Q0

df

In [ ]:
with flopy.plot.styles.USGSPlot():
    fig, axs = plt.subplots(2, 1, figsize=(9, 3), sharex=True)

    fig.suptitle("Leake Spring")

    ax = axs[0]
    # ax.set_ylim(-5, 25)
    df["SPRING-FLOW"].plot(ax=ax, ls="-", marker="o", clip_on=False)
    ax.axhline(0, lw=0.5, color="black")
    ax.set_ylabel("Spring\nDischarge, cfs")

    ax = axs[1]
    ax.set_ylim(-100, 100)
    df["PCT_DIFF"].plot(ax=ax, ls="-", marker="o", clip_on=False)
    ax.axhline(0, lw=0.5, color="black")
    ax.set_ylabel("Reduction\n in Spring\nDischarge, %")
    ax.set_xlabel("Stress Period")

### Streamflow results

In [ ]:
df = riv.output.obs().get_dataframe(start_datetime=start_date_time)
df["RIV-SWGW"] /= -86400
df["TOTAL"] = df["RIV-SWGW"]
Q0 = df["TOTAL"].iloc[0]
df["PCT_DIFF"] = -100.0 * (df["TOTAL"] - Q0) / Q0
df

In [ ]:
with flopy.plot.styles.USGSPlot():
    fig, axs = plt.subplots(2, 1, figsize=(9, 3), sharex=True)

    fig.suptitle("Southern Boundary - Gage 1")

    ax = axs[0]
    ax.set_ylim(-15, 15)
    df["TOTAL"].plot(ax=ax, ls="-", marker="o", clip_on=False)
    ax.axhline(0, lw=0.5, color="black")
    ax.set_ylabel("River\nDischarge, cfs")

    ax = axs[1]
    ax.set_ylim(-100, 200)
    df["PCT_DIFF"].plot(ax=ax, ls="-", marker="o", clip_on=False)
    ax.axhline(0, lw=0.5, color="black")
    ax.set_ylabel("Reduction\n in River\nDischarge, %")
    ax.set_xlabel("Stress Period")

### Lake stage

In [ ]:
fpth = ws / f"{name}.lake.obs.csv"

In [ ]:
lak_df = flopy.utils.Mf6Obs(fpth).get_dataframe(start_datetime=start_date_time)
lak_df

In [ ]:
lak_df["LAKE-STAGE"].min()

In [ ]:
with flopy.plot.styles.USGSPlot():
    fig, ax = plt.subplots(1, 1, figsize=(9, 1.5))

    lak_df["LAKE-STAGE"].plot(
        ax=ax,
        ls="-",
        marker="o",
        clip_on=False,
    )
    ax.axhline(0, lw=0.5, color="black")
    ax.set_ylabel("Lake\nStage, ft")
    ax.set_xlabel("Stress Period")
    ax.set_ylim(-5, 14)

In [ ]:
# ws = pl.Path(f"../../models/synthetic-valley-working-{sample_frequency}")
# sim.set_sim_path(ws)

# gwf = sim.get_model()
# array = gwf.npf.k.array.copy()
# # print(array[0,:,:].max())
# # plt.imshow(array[0,:,:])
# # plt.show()
# for k in range(array.shape[0]):
#     if k == 0:
#         mx = array[k, :, :].max()
#         mask = array[k, :, :] == mx
#         # print(mx)
#         # print(mask)

#     # if k == 2:
#     #    mn = array[k, :, :].min()
#     #    mx = array[k, :, :].max()
#     #    array[k, :, :] = mx
#     #    array[k, : int(array.shape[1] / 4), :] = mn
#     # else:
#     array[k, :, :] = 10 ** (np.log10(array[k, :, :]).mean())
#     if k == 0:
#         arr = array[k, :, :]
#         arr[mask] = mx
#         array[k, :, :] = arr

# gwf.npf.k = array
# plt.imshow(gwf.npf.k.array[0, :, :])
# plt.show()

# array = gwf.npf.k33.array.copy()
# # plt.imshow(array[2,:,:])
# for k in range(array.shape[0]):
#     # if k == 2:
#     #    mn = array[k, :, :].min()
#     #    mx = array[k, :, :].max()
#     #    array[k, :, :] = mx
#     #    print(mn,mx)
#     #    array[k, : int(array.shape[1] / 3), :int(array.shape[1] / 3)] = mn
#     # else:
#     array[k, :, :] = 10 ** (np.log10(array[k, :, :]).mean())
# plt.imshow(array[2, :, :])
# gwf.npf.k33 = array

# sim.write_simulation()
# sim.run_simulation()